In [1]:
import pandas as pd

data = pd.read_csv('games.csv', index_col=0)

In [2]:
# Cleaning / getting data ready for machine learning
data['venue'] = data['venue'].map({'Home' : 1, 'Away' : 0})     # convert venue to 1's and 0's
data['Date'] = pd.to_datetime(data['Date'])
data['opponent'] = data['Opponent'].astype('category').cat.codes # Converting opponent to integers
data = data.drop(columns=['Time'], inplace=False)                # Dropping time column
data = data[data["Team"] != "Arizona Coyotes"]                   # Team doesnt exist anymore

predict = ['venue']

In [3]:
data

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result,opponent
0,2021-12-12,Anaheim Ducks,St. Louis Blues,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1,25
1,2023-01-28,Anaheim Ducks,Arizona Coyotes,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1,1
2,2022-01-14,Anaheim Ducks,Minnesota Wild,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0,14
3,2024-11-05,Anaheim Ducks,Vancouver Canucks,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0,29
4,2022-10-12,Anaheim Ducks,Seattle Kraken,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,Winnipeg Jets,Tampa Bay Lightning,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1,26
10492,2025-03-07,Winnipeg Jets,New Jersey Devils,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1,17
10493,2023-02-20,Winnipeg Jets,New York Rangers,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1,19
10494,2025-03-09,Winnipeg Jets,Carolina Hurricanes,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0,5


In [4]:
# Function to compute rolling averages over the last 5 games. Games for which there isnt enough data (i.e. the first 3 games of each teams) are dropped

def rolling_averages(team, cols, new_cols, window=3):
    team = team.sort_values("Date")    # Getting team data organized chronologically
    rolling = team[cols].rolling(window, closed='left').mean()   # closed=left to ignore current row in sliding window
    team[new_cols] = rolling
    team = team.dropna(subset=new_cols) # dropping first rows because not enough data
    return team

In [5]:
cols = ['G', 'GA', 'S', 'SV%', 'S%']   # wanted columns for rolling
new_cols = [f"{c}_rolling" for c in cols]
predictors = new_cols + predict                         # New predictors


games_data = data.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols, 3))   # Compute rolling averages
games_data = games_data.droplevel("Team")
games_data.index = range(games_data.shape[0])  # fixing index level
games_data

C:\Users\vikto\AppData\Local\Temp\ipykernel_33544\322024329.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games_data = data.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols, 3))   # Compute rolling averages


,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,...,SA%,Opponent SV%,Opponent PIM,Result,opponent,G_rolling,GA_rolling,S_rolling,SV%_rolling,S%_rolling
0,2021-10-19,Anaheim Ducks,Edmonton Oilers,0,14082.0,5,6,36,13.9,0.861,...,16.2,0.733,4,0,11,2.666667,1.666667,26.000000,0.959000,10.900000
1,2021-10-21,Anaheim Ducks,Winnipeg Jets,0,13886.0,1,5,39,2.6,0.846,...,18.5,0.974,6,0,32,3.000000,3.333333,30.666667,0.922333,9.466667
2,2021-10-23,Anaheim Ducks,Minnesota Wild,0,18055.0,3,4,24,12.5,0.889,...,11.1,0.875,10,0,14,3.000000,4.333333,34.000000,0.886667,9.200000
3,2021-10-26,Anaheim Ducks,Winnipeg Jets,1,11951.0,3,4,35,8.6,0.840,...,16.0,0.914,6,0,32,3.000000,5.000000,33.000000,0.865333,9.666667
4,2021-10-28,Anaheim Ducks,Buffalo Sabres,1,12014.0,3,4,37,8.1,0.862,...,13.8,0.919,2,0,3,2.333333,4.333333,32.666667,0.858333,7.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10146,2025-04-07,Winnipeg Jets,St. Louis Blues,1,15225.0,3,1,26,11.5,0.933,...,6.7,0.920,2,1,25,2.000000,2.666667,24.666667,0.910333,8.833333
10147,2025-04-10,Winnipeg Jets,Dallas Stars,0,18532.0,4,0,35,11.4,1.000,...,0.0,0.886,7,1,9,2.666667,1.666667,27.000000,0.940667,10.900000
10148,2025-04-12,Winnipeg Jets,Chicago Blackhawks,0,20634.0,5,4,42,9.5,0.875,...,12.5,0.905,4,1,6,2.666667,1.666667,31.333333,0.940667,8.633333
10149,2025-04-13,Winnipeg Jets,Edmonton Oilers,1,15225.0,1,4,18,5.6,0.921,...,10.3,0.944,4,0,11,4.000000,1.666667,34.333333,0.936000,10.800000


In [6]:
games_data = games_data[
        (games_data['Team'] != 'Arizona Coyotes') &
        (games_data['Opponent'] != 'Arizona Coyotes')
        ]

In [7]:
#####
# 1. Initialize Elo ratings
initial_elo = 1500
teams = games_data['Team'].unique()
elo_ratings = {team: initial_elo for team in teams}

elo_features = []

# 2. Loop through each game and update ratings
for idx, row in games_data.iterrows():
    team = row['Team']
    opponent = row['Opponent']
    venue = row['venue']
    result = row['Result']  # 1 if win, 0 if loss

    # Optional: home-ice advantage
    team_elo = elo_ratings[team] + (35 if venue == 'Home' else 0)
    opponent_elo = elo_ratings[opponent]

    # Store Elo features BEFORE the game
    elo_features.append({
        'team_elo': team_elo,
        'opponent_elo': opponent_elo,
        'elo_diff': team_elo - opponent_elo
    })

    # Calculate expected outcome
    expected_win = 1 / (1 + 10 ** ((opponent_elo - team_elo) / 400))

    # Elo update (K-factor can be tuned)
    k = 30
    change = k * (result - expected_win)
    elo_ratings[team] += change
    elo_ratings[opponent] -= change

# Convert Elo features to DataFrame
elo_df = pd.DataFrame(elo_features)

# Merge with combined_team_view
games_data = pd.concat([games_data.reset_index(drop=True), elo_df], axis=1)

####
new = ['team_elo', 'opponent_elo', 'elo_diff']
predictors = predictors + new

In [8]:
games_data

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,...,Result,opponent,G_rolling,GA_rolling,S_rolling,SV%_rolling,S%_rolling,team_elo,opponent_elo,elo_diff
0,2021-10-19,Anaheim Ducks,Edmonton Oilers,0,14082.0,5,6,36,13.9,0.861,...,0,11,2.666667,1.666667,26.000000,0.959000,10.900000,1500.000000,1500.000000,0.000000
1,2021-10-21,Anaheim Ducks,Winnipeg Jets,0,13886.0,1,5,39,2.6,0.846,...,0,32,3.000000,3.333333,30.666667,0.922333,9.466667,1485.000000,1500.000000,-15.000000
2,2021-10-23,Anaheim Ducks,Minnesota Wild,0,18055.0,3,4,24,12.5,0.889,...,0,14,3.000000,4.333333,34.000000,0.886667,9.200000,1470.647200,1500.000000,-29.352800
3,2021-10-26,Anaheim Ducks,Winnipeg Jets,1,11951.0,3,4,35,8.6,0.840,...,0,32,3.000000,5.000000,33.000000,0.865333,9.666667,1456.911456,1514.352800,-57.441344
4,2021-10-28,Anaheim Ducks,Buffalo Sabres,1,12014.0,3,4,37,8.1,0.862,...,0,3,2.333333,4.333333,32.666667,0.858333,7.900000,1444.369047,1500.000000,-55.630953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9903,2025-04-07,Winnipeg Jets,St. Louis Blues,1,15225.0,3,1,26,11.5,0.933,...,1,25,2.000000,2.666667,24.666667,0.910333,8.833333,1613.819956,1469.038453,144.781503
9904,2025-04-10,Winnipeg Jets,Dallas Stars,0,18532.0,4,0,35,11.4,1.000,...,1,9,2.666667,1.666667,27.000000,0.940667,10.900000,1622.907569,1577.111848,45.795721
9905,2025-04-12,Winnipeg Jets,Chicago Blackhawks,0,20634.0,5,4,42,9.5,0.875,...,1,6,2.666667,1.666667,31.333333,0.940667,8.633333,1635.941781,1338.473488,297.468292
9906,2025-04-13,Winnipeg Jets,Edmonton Oilers,1,15225.0,1,4,18,5.6,0.921,...,0,11,4.000000,1.666667,34.333333,0.936000,10.800000,1640.527496,1544.564759,95.962738


In [9]:
games_data['days_since_last'] = games_data.groupby('Team')['Date'].diff().dt.days
games_data['days_since_last'].fillna(-1, inplace=True)

games_data['goal_diff'] = games_data['G'] - games_data['GA']

test = ['goal_diff']
new_col = ['goal_diff_rolling']

games_data = games_data.groupby('Team').apply(lambda x: rolling_averages(x, test, new_col, 5))
games_data = games_data.droplevel('Team')
games_data.index = range(len(games_data))

games_data

C:\Users\vikto\AppData\Local\Temp\ipykernel_33544\563594204.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  games_data['days_since_last'].fillna(-1, inplace=True)
C:\Users\vikto\AppData\Local\Temp\ipykernel_33544\563594204.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns afte

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,...,GA_rolling,S_rolling,SV%_rolling,S%_rolling,team_elo,opponent_elo,elo_diff,days_since_last,goal_diff,goal_diff_rolling
0,2021-10-29,Anaheim Ducks,Vegas Golden Knights,0,18029.0,4,5,38,10.5,0.905,...,4.000000,32.000000,0.863667,9.733333,1431.750511,1500.000000,-68.249489,1.0,-1,-1.6
1,2021-10-31,Anaheim Ducks,Montreal Canadiens,1,11652.0,4,2,26,15.4,0.889,...,4.333333,36.666667,0.869000,9.066667,1419.659754,1500.000000,-80.340246,2.0,2,-1.6
2,2021-11-02,Anaheim Ducks,New Jersey Devils,1,13252.0,4,0,27,14.8,1.000,...,3.666667,33.666667,0.885333,11.333333,1438.067793,1500.000000,-61.932207,2.0,4,-0.4
3,2021-11-07,Anaheim Ducks,St. Louis Blues,1,12056.0,4,1,27,14.8,0.971,...,1.000000,28.000000,0.951667,13.300000,1455.713656,1500.000000,-44.286344,5.0,3,0.6
4,2021-11-09,Anaheim Ducks,Vancouver Canucks,0,18201.0,3,2,30,10.0,0.953,...,0.666667,28.333333,0.979000,13.100000,1472.615363,1500.000000,-27.384637,2.0,1,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,2025-04-07,Winnipeg Jets,St. Louis Blues,1,15225.0,3,1,26,11.5,0.933,...,2.666667,24.666667,0.910333,8.833333,1613.819956,1469.038453,144.781503,2.0,2,0.8
9744,2025-04-10,Winnipeg Jets,Dallas Stars,0,18532.0,4,0,35,11.4,1.000,...,1.666667,27.000000,0.940667,10.900000,1622.907569,1577.111848,45.795721,3.0,4,0.4
9745,2025-04-12,Winnipeg Jets,Chicago Blackhawks,0,20634.0,5,4,42,9.5,0.875,...,1.666667,31.333333,0.940667,8.633333,1635.941781,1338.473488,297.468292,2.0,1,0.8
9746,2025-04-13,Winnipeg Jets,Edmonton Oilers,1,15225.0,1,4,18,5.6,0.921,...,1.666667,34.333333,0.936000,10.800000,1640.527496,1544.564759,95.962738,1.0,-3,1.6


In [10]:
new_pred = ['days_since_last']
predictors = predictors + new_pred
predictors

['G_rolling',
 'GA_rolling',
 'S_rolling',
 'SV%_rolling',
 'S%_rolling',
 'venue',
 'team_elo',
 'opponent_elo',
 'elo_diff',
 'days_since_last']

In [11]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression

model = XGBClassifier(random_state=10, )    # base model

# Defining Time Series Split
TSS = TimeSeriesSplit(n_splits=5)

test_model = RandomForestClassifier(random_state=10)

lin = BaggingClassifier(LogisticRegression(random_state=10, solver='liblinear', penalty='l2', max_iter = 1000))


In [12]:
from sklearn.metrics import precision_score

# Function to make predictions given the data, input features and chosen model

def make_predictions(data, predictors, model):
    train = data[data['Date'] < '2024-04-19']
    #train = train[train['Date'] > '2022-10-06']
    test = data[data['Date'] > '2024-04-19']
    model.fit(train[predictors], train['Result'])
    preds = model.predict(test[predictors])
    combined  = pd.DataFrame(dict(actual=test['Result'], prediction = preds), index=test.index)
    precision = precision_score(test['Result'], preds)
    return combined, precision

In [28]:
# Defining search space for GridSearchCV
search_grid = {
    'n_estimators': [50, 75, 100, 150],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.02, 0.03],
    'reg_alpha': [1, 5, 10],
    'reg_lambda': [1, 5, 10]

}

alt_search_grid = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [3, 6, 9],
    'min_samples_split': [3, 5, 10]
}

lin_search_grid = {
    # Logistic Regression hyperparameters (base_estimator__)
    'estimator__C' : [0.5, 0.8, 1.0],
    'n_estimators': [10, 50, 100, 150, 200],
}

GS = GridSearchCV(
    estimator = test_model,
    param_grid = alt_search_grid,
    scoring = 'neg_log_loss',
    refit = True,
    cv = TSS,
    verbose= 4
)

training = games_data[games_data['Date'] < '2024-04-19']  # Training using 2021-2024 data
#training = training[training['Date'] > '2022-10-06']
testing = games_data[games_data['Date'] > '2024-04-19']   # Testing on most recent season (2024-2025)


In [29]:
GS.fit(training[predictors], training['Result'])     # Training

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END max_depth=3, min_samples_split=3, n_estimators=50;, score=-0.660 total time=   0.0s
[CV 2/5] END max_depth=3, min_samples_split=3, n_estimators=50;, score=-0.663 total time=   0.0s
[CV 3/5] END max_depth=3, min_samples_split=3, n_estimators=50;, score=-0.673 total time=   0.0s
[CV 4/5] END max_depth=3, min_samples_split=3, n_estimators=50;, score=-0.668 total time=   0.0s
[CV 5/5] END max_depth=3, min_samples_split=3, n_estimators=50;, score=-0.684 total time=   0.0s
[CV 1/5] END max_depth=3, min_samples_split=3, n_estimators=100;, score=-0.658 total time=   0.0s
[CV 2/5] END max_depth=3, min_samples_split=3, n_estimators=100;, score=-0.663 total time=   0.0s
[CV 3/5] END max_depth=3, min_samples_split=3, n_estimators=100;, score=-0.674 total time=   0.0s
[CV 4/5] END max_depth=3, min_samples_split=3, n_estimators=100;, score=-0.667 total time=   0.1s
[CV 5/5] END max_depth=3, min_samples_split=3, n_estimators=1

,estimator,RandomForestC...ndom_state=10)
,param_grid,"{'max_depth': [3, 6, ...], 'min_samples_split': [3, 5, ...], 'n_estimators': [50, 100, ...]}"
,scoring,'neg_log_loss'
,n_jobs,None
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,100


In [30]:
GS.best_score_

np.float64(-0.6690635432579765)

In [31]:
new_model = GS.best_estimator_
new_model

,n_estimators,100
,criterion,'gini'
,max_depth,3
,min_samples_split,3
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [32]:
combined, precision = make_predictions(games_data, predictors, new_model)
precision

0.5536626916524702

In [33]:
from sklearn.metrics import classification_report, roc_auc_score, log_loss
predictions = new_model.predict(testing[predictors])

print(classification_report(testing['Result'], predictions))

              precision    recall  f1-score   support

           0       0.54      0.60      0.57      1308
           1       0.55      0.50      0.52      1308

    accuracy                           0.55      2616
   macro avg       0.55      0.55      0.55      2616
weighted avg       0.55      0.55      0.55      2616



In [34]:
# Create DataFrame pairing features with their importances
importances = pd.DataFrame({
    'Feature': predictors,
    'Importance': new_model.feature_importances_
})

# Sort by importance
importances = importances.sort_values(by='Importance', ascending=False)

# Display top features
print(importances.head(10))


           Feature  Importance
8         elo_diff    0.381957
6         team_elo    0.249775
7     opponent_elo    0.187429
2        S_rolling    0.078191
5            venue    0.022160
4       S%_rolling    0.021060
9  days_since_last    0.020837
1       GA_rolling    0.016186
3      SV%_rolling    0.012151
0        G_rolling    0.010255


In [35]:
combined

,actual,prediction
229,1,0
230,0,0
231,1,0
232,0,0
233,0,0
...,...,...
9743,1,1
9744,1,0
9745,1,1
9746,0,1


In [36]:
combined = combined.merge(games_data[['Date', 'Team', 'Opponent', 'Result']], left_index=True, right_index=True)
combined

,actual,prediction,Date,Team,Opponent,Result
229,1,0,2024-10-12,Anaheim Ducks,San Jose Sharks,1
230,0,0,2024-10-13,Anaheim Ducks,Vegas Golden Knights,0
231,1,0,2024-10-16,Anaheim Ducks,Utah Hockey Club,1
232,0,0,2024-10-18,Anaheim Ducks,Colorado Avalanche,0
233,0,0,2024-10-20,Anaheim Ducks,Los Angeles Kings,0
...,...,...,...,...,...,...
9743,1,1,2025-04-07,Winnipeg Jets,St. Louis Blues,1
9744,1,0,2025-04-10,Winnipeg Jets,Dallas Stars,1
9745,1,1,2025-04-12,Winnipeg Jets,Chicago Blackhawks,1
9746,0,1,2025-04-13,Winnipeg Jets,Edmonton Oilers,0


In [37]:
final = combined.merge(combined, left_on=['Date', 'Team'], right_on=['Date', 'Opponent'])  # few games will drop due to rolling windows
final

,actual_x,prediction_x,Date,Team_x,Opponent_x,Result_x,actual_y,prediction_y,Team_y,Opponent_y,Result_y
0,1,0,2024-10-12,Anaheim Ducks,San Jose Sharks,1,0,0,San Jose Sharks,Anaheim Ducks,0
1,0,0,2024-10-13,Anaheim Ducks,Vegas Golden Knights,0,1,1,Vegas Golden Knights,Anaheim Ducks,1
2,0,0,2024-10-18,Anaheim Ducks,Colorado Avalanche,0,1,1,Colorado Avalanche,Anaheim Ducks,1
3,0,0,2024-10-20,Anaheim Ducks,Los Angeles Kings,0,1,1,Los Angeles Kings,Anaheim Ducks,1
4,1,0,2024-10-22,Anaheim Ducks,San Jose Sharks,1,0,0,San Jose Sharks,Anaheim Ducks,0
...,...,...,...,...,...,...,...,...,...,...,...
2603,1,1,2025-04-07,Winnipeg Jets,St. Louis Blues,1,0,0,St. Louis Blues,Winnipeg Jets,0
2604,1,0,2025-04-10,Winnipeg Jets,Dallas Stars,1,0,0,Dallas Stars,Winnipeg Jets,0
2605,1,1,2025-04-12,Winnipeg Jets,Chicago Blackhawks,1,0,0,Chicago Blackhawks,Winnipeg Jets,0
2606,0,1,2025-04-13,Winnipeg Jets,Edmonton Oilers,0,1,0,Edmonton Oilers,Winnipeg Jets,1


In [38]:
final[(final['prediction_x'] == 1) & (final['prediction_y'] == 0)]['actual_x'].value_counts()

actual_x
1    542
0    415
Name: count, dtype: int64

In [39]:
final[(final['prediction_x'] == 0) & (final['prediction_y'] == 1)]['actual_y'].value_counts()

actual_y
1    542
0    415
Name: count, dtype: int64

In [40]:
print("Overall accuracy turned out to be ~60%.")
print("However, when merging predictions for both games, we can see that the model is 63.3% accurate when both teams predictions match.")

Overall accuracy turned out to be ~60%.
However, when merging predictions for both games, we can see that the model is 63.3% accurate when both teams predictions match.


In [26]:
total = 560 + 390
total

950

In [27]:
560/total

0.5894736842105263